In [4]:
pip install ipyaladin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.4/787.4 kB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [6]:
# the catalogue name in VizieR
CATALOGUE = "J/ApJ/940/63"

We first retrieve the catalogue information.

In [7]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [8]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Reliable distances of Galactic SNRs
Short Name: J/ApJ/940/63
IVOA Identifier: ivo://cds.vizier/j/apj/940/63
Access modes: conesearch, tap#aux, web
- tap#aux: http://tapvizier.cds.unistra.fr/TAPVizieR/tap
- webpage: http://vizier.cds.unistra.fr/viz-bin/VizieR-2?-source=J/ApJ/940/63
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/ApJ/940/63/table1?, description: Cone search capability for
 table J/ApJ/940/63/table1 (Galactic supernova remnants (SNRs) with reliable
 distances)

We present a table of 215 supernova remnants (SNRs) with distances. New
distances are found to SNR G51.26+0.11 of 6.6+/-1.7kpc using HI absorption
spectra and to five other SNRs using maser/molecular cloud associations. We
recalculate the distances and errors to all SNRs using a consistent rotation
curve and provide errors where they were not previously estimated. This
results in a significant distance revisions for 20 SNRs. Because of
observational constraints and selection effects, there is an 

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [9]:
voresource.creators[0]

'Ranasinghe S.'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [10]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In this catalogue, we have 2 tables.
J/ApJ/940/63/table1: Galactic supernova remnants (SNRs) with reliable distances
J/ApJ/940/63/refs: References


In [11]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/ApJ/940/63/table1', 'J/ApJ/940/63/refs']

In [15]:
import pandas as pd
from astropy.table import Table

In [ ]:
# Dictionary to store Pandas DataFrames
dataframes = {}



📌 Processing Table: J/ApJ/940/63/table1
❌ Error converting table J/ApJ/940/63/table1: 'list' object has no attribute 'keys'

📌 Processing Table: J/ApJ/940/63/refs
❌ Error converting table J/ApJ/940/63/refs: 'list' object has no attribute 'keys'


In [ ]:
tables['J/ApJ/940/63/table1'].

''

In [26]:
type(tables['J/ApJ/940/63/table1'])

pyvo.io.vosi.vodataservice.VODataServiceTable

The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [12]:
voresource.access_modes()

{'conesearch', 'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [13]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJ/940/63'

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [ ]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Finding the conesearch service that you want to use (there is usually one per table):

Let's use the first one for this example. 

In [ ]:
# we get the conesearch  service associated to the first table
conesearch_interface = voresource.get_interface(service_type='conesearch', 
                                                keyword='J/ApJ/940/63/table1',
                                                lax=True)
# if you get a TypeError about an unexpected keyword, check that you installed pyvo>=1.6
conesearch_service = conesearch_interface.to_service()

We adapt the radius and position of the center to our needs:

In [ ]:
conesearch_radius = 1 / 60.0  # in degrees
conesearch_center = (266.420800, -29.013300)

In [ ]:
conesearch_records = conesearch_service.search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

In [ ]:
# retrieve the MOC
catalogue_coverage = MOC.from_vizier_table(CATALOGUE)
catalogue_coverage.display_preview()

The quick preview is the whole sky in equatorial coordinates. Red represent zones where the catalog has data, black areas are empty zones.

We can also plot the coverage with ``matplotlib``.

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = catalogue_coverage.wcs(fig)
ax = fig.add_subplot(projection=wcs)
catalogue_coverage.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")

or with [ipyaladin](https://github.com/cds-astro/ipyaladin)

In [ ]:
aladin = Aladin()
aladin

We switch the target to the center of the conesearch

In [ ]:
aladin.target = "266.420800 -29.013300"

We can add the tables and coverage in this ipyaladin widget:

In [ ]:
aladin.add_table(conesearch_records.to_table(), color="lightskyblue", shape="plus")

If you don't see the points from the conesearch result, try zooming in the widget with your mouse wheel.

We can also add the result of the TAP query:

In [ ]:
aladin.add_table(tap_records.to_table())

In [ ]:
aladin.add_moc(
    catalogue_coverage,
    fill=True, opacity=0.5, color="pink", edge=True
)

For more information about manipulating multi-order coverages, see [MOCpy documentation](https://cds-astro.github.io/mocpy/), and about using the `ipyaladin` widget, see [its documentation](https://cds-astro.github.io/ipyaladin/).